In [ ]:
! pwd

In [1]:
import os
import json

In [2]:
path = "../src/utils/new_filter_data.js"

In [3]:
with open(path) as f:
    text = f.read().split("\n")

In [10]:
dictionary = eval(text[1][:-1])

In [13]:
from typing import Dict, Any, List, Optional

class TreeNode:
    """
    Represents a single node in the hierarchical tree structure.
    """
    def __init__(
        self,
        id: str,
        label: str,
        category: str,
        primary_group: str,
        count: str,
        description: str,
        children: Optional[List['TreeNode']] = None,
    ):
        """
        Initializes a new tree node with data attributes.

        Args:
            id (str): The unique identifier for the node.
            label (str): The human-readable label or name.
            category (str): The category of the node.
            primary_group (str): The primary group classification.
            count (str): A count associated with the node.
            children (Optional[List['TreeNode']]): A list of child TreeNode objects.
        """
        self.id = id
        self.label = label
        self.category = category
        self.primary_group = primary_group
        self.description = description
        self.count = count
        self.children: List['TreeNode'] = children if children is not None else []

    def __repr__(self):
        """
        Provides a string representation for debugging.
        """
        return f"TreeNode(id='{self.id}', label='{self.label}', children={len(self.children)})"

    def print_tree(self, level=0):
        """
        Recursively prints the structure of the tree.
        """
        indent = "  " * level
        print(f"{indent}- {self.label} (ID: {self.id})")
        for child in self.children:
            child.print_tree(level + 1)

In [28]:
def build_dict(tree: TreeNode) -> dict:
    "Recursively builds the new dictionary from the tree"

    dictionary = {}
    for label in ["id", "label", "category", "primary_group", "count"]:
        dictionary[label] = getattr(tree, label)
    if tree.children:
        dictionary["children"] = [build_dict(child) for child in tree.children]
    return dictionary
    

In [14]:
def build_tree(data: Dict[str, Any]) -> TreeNode:
    """
    Recursively builds the TreeNode structure from the raw dictionary data.

    Args:
        data (Dict[str, Any]): The raw dictionary data for a single node.

    Returns:
        TreeNode: The resulting object-oriented tree node.
    """
    # Extract core attributes
    node_id = data.get('id', '')
    label = data.get('label', '')
    category = data.get('category', '')
    primary_group = data.get('primaryGroup', '')
    count = data.get('count', '')
    description = data.get('description', '')

    # Initialize the current node
    current_node = TreeNode(
        id=node_id,
        label=label,
        category=category,
        primary_group=primary_group,
        count=count,
        description=description
    )

    # Check for children and recurse
    raw_children = data.get('children', {})
    if raw_children and isinstance(raw_children, dict):
        # Iterate over the values (the child dictionaries)
        for child_dict in raw_children.values():
            # Recursively build the child node
            child_node = build_tree(child_dict)
            current_node.children.append(child_node)

    return current_node

In [15]:
tree = build_tree(dictionary['0_0'])

In [19]:
tcga = tree.children.pop()

In [23]:
for child in tcga.children:
    child.label = f"{child.label} ({child.description})"

In [25]:
tree.children.append(tcga)

In [29]:
new_dictionary = build_dict(tree)

In [35]:
new_dictionary["children"][2]

{'id': '0_0_2',
 'label': 'crukTerms',
 'category': 'cancerTypes',
 'primary_group': '',
 'count': '0',
 'children': [{'id': '0_0_2_0',
   'label': 'All',
   'category': 'crukTerms',
   'primary_group': '',
   'count': '0'},
  {'id': '0_0_2_1',
   'label': 'Acute lymphoblastic leukaemia (ALL)',
   'category': 'crukTerms',
   'primary_group': '',
   'count': '0'},
  {'id': '0_0_2_2',
   'label': 'Acute lymphoblastic leukaemia (ALL) in children',
   'category': 'crukTerms',
   'primary_group': '',
   'count': '0'},
  {'id': '0_0_2_3',
   'label': 'Acute myeloid leukaemia (AML)',
   'category': 'crukTerms',
   'primary_group': '',
   'count': '0'},
  {'id': '0_0_2_4',
   'label': 'Adrenal gland tumours',
   'category': 'crukTerms',
   'primary_group': '',
   'count': '0'},
  {'id': '0_0_2_5',
   'label': 'Ampullary cancer',
   'category': 'crukTerms',
   'primary_group': '',
   'count': '0'},
  {'id': '0_0_2_6',
   'label': 'Anal cancer',
   'category': 'crukTerms',
   'primary_group': ''

In [38]:
print("a\nb")

a
b


In [36]:
dictionary['0_0'] = new_dictionary

In [41]:
new_text = text[0]+"\n"+str(dictionary)+"\n"+text[2]+";"

In [42]:
path = "../src/utils/longer_filter_data.js"

In [44]:
with open(path, "w") as f:
    f.write(new_text)